In [1]:
import openai
import pandas as pd
import numpy as np
import ast
from getpass import getpass
import os

openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
#embedding function without the file path stuff
# Load your CSV file
data = pd.read_csv('./FOMC minutes csv cleaned/fed_speech_may1.csv')


# Function to fetch embeddings from OpenAI
def get_embeddings(text):
    try:
        response = openai.Embedding.create(
            input=text,
            engine="text-embedding-3-small"  # Check the latest documentation for available engines
        )
        return response['data'][0]['embedding']  # Extracting the embedding vector
    except Exception as e:
        return str(e)  # Return error as string for debugging

# Apply the function to each text entry in the CSV and create a new column for embeddings
data['embeddings'] = data['text'].apply(get_embeddings)

# Save the DataFrame with the embeddings to a new CSV file
data.to_csv('FedSpeechMay1_with_embeddings.csv', index=False)

# Printing a message to confirm completion
print("CSV file with embeddings saved successfully.")

FileNotFoundError: [Errno 2] No such file or directory: './FOMC minutes csv cleaned/fed_speech_may1.csv'

In [3]:
# Load the CSV file
data = pd.read_csv('./FedSpeechMay1_with_embeddings.csv')

# Convert string representations of lists to actual lists of floats
data['embeddings'] = data['embeddings'].apply(ast.literal_eval)

# Optional: Convert lists to NumPy arrays for easier numerical manipulation
data['embeddings'] = data['embeddings'].apply(np.array)

# Print the first few rows to check the embeddings

data.to_csv('Updated2Speech.csv', index=False)
print('embedding column now in array format')

embedding column now in array format


In [4]:
type(data.iloc[0].embeddings)

numpy.ndarray

In [13]:
search_term = input('Enter a Search Term:')
search_term_vector = get_embeddings(search_term)
search_term_vector

Enter a Search Term: ekomony


[0.060903389006853104,
 0.01466291956603527,
 0.010025469586253166,
 0.03862754628062248,
 0.08132961392402649,
 -0.010367247276008129,
 0.01628468744456768,
 0.07344862818717957,
 -0.024085253477096558,
 0.01133226603269577,
 0.0399678498506546,
 -0.05757943540811539,
 -0.021552080288529396,
 -0.02093554101884365,
 0.02841443382203579,
 0.028789717704057693,
 -0.02921861596405506,
 -0.03385606408119202,
 -0.0027610252145677805,
 0.03302507847547531,
 0.01660636067390442,
 0.0007082666270434856,
 -0.0007761194719932973,
 -0.005689588375389576,
 -0.007827372290194035,
 0.011265250854194164,
 -0.021096376702189445,
 0.029701124876737595,
 0.02065407671034336,
 -0.0572577640414238,
 0.01221686601638794,
 -0.03567887842655182,
 0.03677792847156525,
 -0.014381456188857555,
 0.007431982550770044,
 -0.017651796340942383,
 -0.01527945976704359,
 0.02908458560705185,
 0.04522183910012245,
 -0.002509718295186758,
 -0.017209496349096298,
 -0.033614810556173325,
 0.004235358908772469,
 -0.01704866

In [14]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0  # Avoid division by zero
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

data['similarities'] = data['embeddings'].apply(lambda x: cosine_similarity(x, search_term_vector))

# Displaying the DataFrame
print(data.head(5))

                                                                                                                                      text  \
0        May 1, 2024  Chair Powell’s Press Conference  PRELIMINARY  Transcript of Chair Powell’s Press Conference May 1, 2024 CHAIR POWELL   
1  My colleagues and I remain squarely focused on our dual mandate to promote maximum employment and stable prices for the American people   
2                                                            The economy has made considerable progress toward our dual mandate objectives   
3                Inflation has eased substantially over the past year while the labor market has remained strong and that’s very good news   
4                  But inflation is still too high, further progress in bringing it down is not assured, and the path forward is uncertain   

                                                                                                                                                   

In [15]:
pd.set_option('display.max_colwidth', None)
data.sort_values("similarities", ascending=False).text[:20]

14                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               economy over the past year
98                                                                                                                                                                                                                                                                                                                                                                                                                   You don't really see that happening, what you see is economic activity 

In [16]:
data.sort_values("similarities", ascending=False).similarities[:20]

14     0.337232
98     0.247765
46     0.235993
198    0.232888
2      0.221538
200    0.209209
201    0.207079
11     0.198111
299    0.194848
202    0.194155
236    0.193791
215    0.192696
47     0.189625
148    0.188593
15     0.187935
234    0.182851
261    0.182178
178    0.182133
228    0.182126
110    0.179846
Name: similarities, dtype: float64